In [ ]:
# ProjectQ quantum circuit library 
import projectq.libs.math
import projectq.setups.decompositions
from projectq.backends import Simulator, ResourceCounter
from projectq.cengines import (AutoReplacer, DecompositionRuleSet,
                               InstructionFilter, LocalOptimizer,
                               MainEngine, TagRemover)
from projectq.libs.math import (AddConstant, AddConstantModN,
                                MultiplyByConstantModN)
from projectq.meta import Control
from projectq.ops import (All, BasicMathGate, get_inverse, H, Measure, QFT, R,
                          Swap, X)

# 3 non-quantum library
import math
import random
from fractions import Fraction

In [ ]:
# basic gcd
def gcd(a,b):
    if(b==0):
        return int(a)
    return int(gcd(b,a%b))

# basic a^n mod m
def pow_mod(a,n,k):
    # base case
    if(n<=1):
        return int(math.pow(a,n)%k)
    # recursive case
    if(n%2==0):
        tmp = pow_mod(a,n//2,k)
        return int((tmp * tmp) % k)
    else:
        tmp = pow_mod(a,n//2,k)
        return int((((tmp * tmp) % k) * a) % k)

# basic a^n
def power(a,n):
    # base case
    if(n<=1):
        return int(math.pow(a,n))
    # recursive case
    if(n%2==0):
        tmp = power(a,n//2)
        return int(tmp * tmp)
    else:
        tmp = power(a,n//2)
        return int((tmp * tmp) * a)

In [ ]:
"""
As said in ProjectQ webpage --> 
"Simulating Shor s algorithm at the level of single-qubits gates and CNOTs already takes
quite a big of time for larger numbers than 15."
Returning TRUE allows to use the emulation feature, which does not decompose the modular arithmetic to low-level gates.
"""
def high_level_gates(eng, cmd):
    return True

def order_finding2(eng, N, a):
    
    n = int(math.ceil(math.log(N, 2))) # log2(N)

    x = eng.allocate_qureg(n)

    '''Pauli-X gate: acts on a single qubit. It is the quantum equivalent of the NOT gate for classical computers'''
    X | x[0]

    measurements = [0] * (2 * n)  # will hold the 2n measurement results

    ctrl_qubit = eng.allocate_qubit()

    for k in range(2 * n):
        '''Compute a^x mod N conditioned on a control qubit ctrl_qubit in a uniform superposition of 0 and 1'''
        current_a = pow_mod(a, 1 << (2 * n - 1 - k), N) # << is binary left shift, so we are computing a^(2^(2n-1-k)) mod N
        # one iteration of 1-qubit QPE (Quantum Phase Estimation)
        H | ctrl_qubit
        with Control(eng, ctrl_qubit):
            MultiplyByConstantModN(current_a, N) | x

        # perform semi-classical inverse QFT --> Rotations conditioned on previous outcomes
        for i in range(k):
            if measurements[i]:
                R(-math.pi/(1 << (k - i))) | ctrl_qubit
        
        # final Hadamard of the inverse QFT
        H | ctrl_qubit

        # and measure
        Measure | ctrl_qubit
        eng.flush() # flush all gates and execute measurements
        measurements[k] = int(ctrl_qubit)
        if measurements[k]:
            X | ctrl_qubit # reset

    All(Measure) | x # shortcut (instance of) projectq.ops.Tensor
    # turn the measured values into a number in [0,1)
    y = sum([(measurements[2 * n - 1 - i]*1. / (1 << (i + 1)))
             for i in range(2 * n)])

    # continued fraction expansion to get denominator (the period)
    r = Fraction(y).limit_denominator(N-1).denominator

    # return the (potential) period
    return r

In [ ]:
"""
PROJECTQ SHOR
"""

# build compilation engine list
rule_set = DecompositionRuleSet(modules=[projectq.libs.math,
                                             projectq.setups.decompositions])
compilerengines = [AutoReplacer(rule_set),
                       InstructionFilter(high_level_gates),
                       TagRemover(),
                       LocalOptimizer(3),
                       AutoReplacer(rule_set),
                       TagRemover(),
                       LocalOptimizer(3)]
# mmake the compiler and run the circuit on the simulator backend
eng = MainEngine(Simulator(), compilerengines)

def ProjectQShorSimulator(N):
    ans0=0
    ans1=0
    iter = 0
    status="failed"
    while(True):
        a = random.randrange(2,N)
        k = gcd(a,N)
        if(k != 1):
            ans0=k
            ans1=N//ans0
            # check founded
            # status = "founded"
            # print("attempt",iter,":",a,status)
            return ans0, ans1 
        else:
            r = order_finding2(eng, N, a)
            g = power(a,r//2)
            if(r%2==0 and ((g+1)%N!=0) and N%(g+1)):
                ans0=gcd(g+1,N)
                ans1=gcd(g-1,N)
                # check founded
                # status = "founded"
                # print("attempt",iter,":",a,status)
                return ans0, ans1 
        # check number of attemp
        # print("attempt",iter,":",a,status)
        iter+=1

# Main program
p=3
q=5
N = p*q
P_pq,Q_pq=ProjectQShorSimulator(N)

# Returning answer
print("--"*10)
print("factor of N =",N)
print("p =",P_pq)
print("q =",Q_pq)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bar

def plot_histogram_of_pq(f, N, n_samples):
    # Parameters for data collection
    P_values = []
    Q_values = []

    # Collect data with progress bar
    print("Collecting probability samples...")
    for _ in tqdm(range(n_samples)):
        P,Q = f(N)
        P_values.append(P)
        Q_values.append(Q)

    # Convert to numpy arrays for easier analysis
    P_values = np.array(P_values)
    Q_values = np.array(Q_values)

    # Create figure
    plt.figure(figsize=(12, 6))

    # Plot histograms
    plt.hist(P_values, bins=100, alpha=0.6, color='blue', label='P', density=True)
    plt.hist(Q_values, bins=100, alpha=0.6, color='red', label='Q', density=True)

    # Customize the plot
    plt.title(f'Distribution of Probabilities P and Q (n={n_samples})', fontsize=12)
    plt.xlabel('Probability Value', fontsize=10)
    plt.ylabel('Density', fontsize=10)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)

    # Display statistics
    print("\nStatistical Summary:")
    print("\nP Statistics:")
    print(f"Mean: {np.mean(P_values):.3f}")
    print(f"Std Dev: {np.std(P_values):.3f}")
    print(f"Min: {np.min(P_values):.3f}")
    print(f"Max: {np.max(P_values):.3f}")

    print("\nQ Statistics:")
    print(f"Mean: {np.mean(Q_values):.3f}")
    print(f"Std Dev: {np.std(Q_values):.3f}")
    print(f"Min: {np.min(Q_values):.3f}")
    print(f"Max: {np.max(Q_values):.3f}")

    plt.show()
    # Save the data (optional)
    # np.savez('probability_data.npz', P_values=P_values, Q_values=Q_values)